This is simply a sandbox for exploring data that we have captured from the inmotion robot in the sensory shift experiment.

In [ ]:
%pylab inline

import pandas as pd
import pickle
import json
import seaborn as sns

In [ ]:
#prefix = "incoming/tim/tim_sensoryshift_06_02.12h54m16"
prefix = "incoming/neeraj_sensoryshift_06_02.14h08m54"

In [ ]:
dumpf = '%sdump.pickle'%prefix
dump = pickle.load(open(dumpf,'rb'))

In [ ]:
print("Read %d trials"%len(dump))

In [ ]:
jsonf = '%sparameters.json'%prefix
conf = json.load(open(jsonf,'r'))

In [ ]:
for i,d in enumerate(dump):
    print("trial %d : %d captured samples"%(i,len(d['captured'])))
    #print(len(d['captured']))

In [ ]:
# Let's show one trial
trial = dump[0]

In [ ]:
trial['mov.direction']

In [ ]:
rec = pd.DataFrame(trial['captured'])
rec.columns = ['x','y','fx','fy','fz']

In [ ]:
rec.head()

In [ ]:
#plot(rec['x']) #???
def draw_circle(ax,pos,rad):
    angs = linspace(0,2*pi,100) # draw a little ball
    x,y=pos
    xs = [ x+rad*cos(a) for a in angs]
    ys = [ y+rad*sin(a) for a in angs]
    plot( xs,ys, '-',color='darkgray')

def draw_reference(ax,conf,trialdata):
    draw_circle(ax,conf['robot_center'],conf['center_marker_radius'])
    draw_circle(ax,trialdata['target_position'],conf['target_radius'])

In [ ]:
from ipywidgets import *

In [ ]:
def plottrial(i):
    f,ax= subplots(1,1,figsize=(12,7))
    trial = dump[i]
    rec = pd.DataFrame(trial['captured'])
    rec.columns = ['x','y','fx','fy','fz']
    draw_reference(ax,conf,trial)
    ax.plot(rec['x'],rec['y'],'-')
    sc = conf['review_force_scale']
    for i in range(0,rec.shape[0],10): # subsample a little bit
        row = rec.iloc[i]
        ax.arrow(row['x'],row['y'],sc*row['fx'],sc*row['fy'])
    ax.set_xlim(-.3,.3)
    ax.set_ylim(-.1,.2)
    ax.set_aspect('equal')
    sns.despine(offset=5)
interact(plottrial,i=(0,len(dump)-1))